In [3]:
"""
Read .bin files coming from GENEActiv devices and save them as a pd.DataFrame in parquet format.

This module is based on the code from the scikit-digital-health python package (https://github.com/pfizer-opensource/scikit-digital-health)

IMP: scikit-digital-health create conflicts with versions needed for other packages. To solve this:
1. I tried to just copy the code can be copied and modified it to work as a standalone module -- however, scikit-digital-health 
    uses a lot of other modules (also implemented in .c) and it is not clear how to separate them without breaking the code.
2. Solution for now: he scikit-digital-health package is installed in a separate conda environment and the code is run from there.

"""

import numpy as np
import pandas as pd
import os
from skdh.io import ReadBin

def bin2parquet(file_path):
    """
    Read a .bin file from a GENEActiv device.
    """
    reader = ReadBin()
    data = reader.predict(file = file_path)
    data_for_df = np.concatenate((data["accel"], data["temperature"].reshape(-1, 1), data["light"].reshape(-1, 1)), axis = 1)
    acc_df = pd.DataFrame(data_for_df, 
                          columns = ["x", "y", "z", "temperature", "light"],
                          index = pd.to_datetime(data["time"], unit = "s"))
    # save as parquet
    acc_df.to_parquet(file_path.replace(".bin", ".parquet"))

if __name__ == "__main__":
    data_path = "/Users/augenpro/Documents/Age-IT/data/Bronze/" # path to the folder containing the subjects
    # participants = sorted([p for p in os.listdir(data_path) if not p.startswith(".")]) # list of the participants
    participants = ["08667", "20603", "36765"]
    visit = "T0 (baseline)" # T0 (baseline), T1 (follow-up @ 6 months), T2 (follow-up @ 12 months)

    sensors = ["GeneActivPolso", "GeneActivCaviglia"]

    for participant in participants:
        print(participant)
        for sensor in sensors:
            path = os.path.join(data_path, participant, visit, sensor)
            files = os.listdir(os.path.join(data_path, participant, visit, sensor))
            # if there are already parquet files, skip
            # if any([f.endswith(".parquet") for f in files]):
            #     continue
            for f in files:
                if f.endswith(".bin"):
                    reader = ReadBin()
                    data = reader.predict(file = os.path.join(path, f))
                    bin2parquet(os.path.join(path, f))

08667


/opt/anaconda3/envs/scikit-digital-health/lib/python3.12/site-packages/skdh/io/base.py:111: UserWarning: Timestamps are local but naive, and no time-zone information is available. This may mean that if a DST change occurs during the recording period, the times will be offset by an hour
  warn(
/opt/anaconda3/envs/scikit-digital-health/lib/python3.12/site-packages/skdh/io/base.py:111: UserWarning: Timestamps are local but naive, and no time-zone information is available. This may mean that if a DST change occurs during the recording period, the times will be offset by an hour
  warn(


20603


/opt/anaconda3/envs/scikit-digital-health/lib/python3.12/site-packages/skdh/io/base.py:111: UserWarning: Timestamps are local but naive, and no time-zone information is available. This may mean that if a DST change occurs during the recording period, the times will be offset by an hour
  warn(
/opt/anaconda3/envs/scikit-digital-health/lib/python3.12/site-packages/skdh/io/base.py:111: UserWarning: Timestamps are local but naive, and no time-zone information is available. This may mean that if a DST change occurs during the recording period, the times will be offset by an hour
  warn(


36765


/opt/anaconda3/envs/scikit-digital-health/lib/python3.12/site-packages/skdh/io/base.py:111: UserWarning: Timestamps are local but naive, and no time-zone information is available. This may mean that if a DST change occurs during the recording period, the times will be offset by an hour
  warn(
/opt/anaconda3/envs/scikit-digital-health/lib/python3.12/site-packages/skdh/io/base.py:111: UserWarning: Timestamps are local but naive, and no time-zone information is available. This may mean that if a DST change occurs during the recording period, the times will be offset by an hour
  warn(


In [ ]:
acc_df = pd.DataFrame([data["accel"], data[""]

((59925000, 3), (59925000,))

In [17]:
np.concatenate((data["accel"], data["temperature"], data["light"])).shape

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [18]:
data["accel"].shape, data["temperature"].shape, data["light"].shape

((59925000, 3), (59925000,), (59925000,))

In [21]:
np.concatenate((data["accel"], data["temperature"].reshape(-1, 1), data["light"].reshape(-1, 1)), axis = 1).shape

(59925000, 5)